# Check predictions

Creates an excel file with all predictions that have the confidence score for target object higher than 0.5

The excel file contains the following columns:
- ImageName
- BoundingBox
- lat
- lon
- yaw
- timestampImage
- timestampDetection

In [52]:
import json
import glob
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from draw_utils import get_exif_data, get_lat_lon
from PIL import Image
import re
from datetime import datetime

images_path = 'D:/DCIM/100MEDIA'

def extract_exif_and_gimbal_yaw(image_path):
    img_exif = Image.open(image_path)
    exif_data = get_exif_data(img_exif)
    lat, lon = get_lat_lon(exif_data)

    timestampImage = exif_data.get('DateTimeOriginal') or exif_data.get('DateTime')
    timestampImage = datetime.strptime(timestampImage, "%Y:%m:%d %H:%M:%S").strftime("%Y-%m-%d_%H:%M:%S")

    with open(image_path, encoding='latin-1') as fd:
        d = fd.read()
        xmp_start = d.find('<x:xmpmeta')
        xmp_end = d.find('</x:xmpmeta')
        xmp_str = d[xmp_start:xmp_end+12]
        match = re.search(r'drone-dji:GimbalYawDegree="([+-][/d.]+)"', xmp_str)
        gimbal_yaw = float(match.group(1)) if match else None

    return lat, lon, gimbal_yaw, timestamp

os.makedirs('./labeled', exist_ok=True)

# Read predictions from JSON file
with open('./predictions_combined.json', 'r') as f:
    predictions = json.load(f)

# Get all .jpg images in the ./detection folder
image_files = glob.glob(f'{images_path}/*.JPG')

image_info = []

# Loop through each image
for img_path in image_files:
    img_name = os.path.basename(img_path)
    image = cv2.imread(img_path)

    for pred in predictions:
        if pred.get('image_id') == img_name[:-4]:
            if pred.get('score') > 0.5 and pred.get('category_id') == 0:
                x, y, w, h = np.array(pred.get('bbox'), dtype=int)
                x_min, y_min = x, y
                x_max, y_max = x + w, y + h

                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 20)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                save_path = os.path.join('./labeled', img_name)
                plt.imsave(save_path, image_rgb)

                img_path = os.path.join(f'{images_path}', img_name)

                try:
                    lat, lon, yaw, timestamp = extract_exif_and_gimbal_yaw(img_path)
                except Exception as e:
                    lat, lon, yaw = None, None, None

                timestampDetection = pred.get('timestampDetection')

                image_info.append({'ImageName': img_name, 'BoundingBox': pred.get('bbox'), 'lat': lat, 'lon': lon, 'yaw': yaw, 'timestampImage': timestamp, 'timestampDetection': timestampDetection})

info_df = pd.DataFrame(image_info)
info_df.to_excel('image_info.xlsx', index=False)

# Get coordinates of labels

Get coordinates of labels in the format image name, lat, lon and timestamp

The excel file contains the following columns:
- ImageName
- BBoxCenterLat
- BBoxCenterLon
- TimestampImage

In [ ]:
import os
from PIL import Image
from draw_utils import get_exif_data, get_lat_lon
import pandas as pd
import numpy as np
import cv2

import matplotlib.pyplot as plt

def get_image_corners(img_width, img_height):
    # Returns corners in image coordinates (centered at image center)
    half_w = img_width / 2
    half_h = img_height / 2
    return np.array([
        [-half_w, -half_h],  # top-left
        [ half_w, -half_h],  # top-right
        [ half_w,  half_h],  # bottom-right
        [-half_w,  half_h],  # bottom-left
    ])

def rotate_points(points, angle_deg):
    # Rotates points by angle_deg (counterclockwise)
    theta = np.deg2rad(angle_deg)
    rot_mat = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])
    return points @ rot_mat.T

def image_corners_to_latlon(center_lat, center_lon, yaw_deg, img_width, img_height, gsd):
    # gsd: ground sample distance in meters per pixel
    # Returns lat/lon for each corner
    # Approximate: assumes flat earth and small area
    corners = get_image_corners(img_width, img_height)
    rotated = rotate_points(corners, yaw_deg)
    # Convert meters to lat/lon offsets
    # 1 deg latitude ~ 111320 m, 1 deg longitude ~ 111320*cos(lat) m
    dlat = rotated[:,1] * gsd / 111320
    dlon = rotated[:,0] * gsd / (111320 * np.cos(np.deg2rad(center_lat)))
    lats = center_lat + dlat
    lons = center_lon + dlon
    return lats, lons

def calculate_bbox_center_latlon(center_lat, center_lon, yaw, img_width, img_height, gsd, bbox_str):
    # Parse bbox string
    x_min, y_min, w, h = [float(v) for v in bbox_str.strip('[]').split(',')]
    x_max = x_min + w
    y_max = y_min + h
    # Get bbox center in image coordinates
    bbox_center = np.array([
        x_min + (x_max - x_min) / 2,
        y_min + (y_max - y_min) / 2
    ])
    # Center at image center
    bbox_center_centered = bbox_center - np.array([img_width/2, img_height/2])
    # Rotate by yaw
    bbox_center_rot = rotate_points(bbox_center_centered.reshape(1, 2), yaw)[0]
    # Convert bbox center to lat/lon
    dlat = bbox_center_rot[1] * gsd / 111320
    dlon = bbox_center_rot[0] * gsd / (111320 * np.cos(np.deg2rad(center_lat)))
    # Adjust sign based on position relative to image center
    if bbox_center[1] < img_height / 2:
        dlat = -dlat
    if bbox_center[0] < img_width / 2:
        dlon = -dlon
    bbox_center_lat = center_lat + dlat
    bbox_center_lon = center_lon + dlon
    return bbox_center_lat, bbox_center_lon

gsd = 0.02
df = pd.read_excel('image_info.xlsx')

# Loop over all images in df
bbox_centers_latlon = []
for idx, row in df.iterrows():
    img_name = row['ImageName']
    bbox_str = row['BoundingBox']
    img_timestamp = row.get('timestampImage', None)

    info_row = info_df[info_df['ImageName'] == img_name]
    if info_row.empty:
        bbox_centers_latlon.append((np.nan, np.nan))
        continue

    center_lat = info_row.iloc[0]['lat']
    center_lon = info_row.iloc[0]['lon']
    yaw = info_row.iloc[0]['yaw']
    img_path = os.path.join('./dataset/images/test', img_name)
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]
    bbox_center_lat, bbox_center_lon = calculate_bbox_center_latlon(center_lat, center_lon, yaw, img_width, img_height, gsd, bbox_str)
    bbox_centers_latlon.append((bbox_center_lat, bbox_center_lon, img_timestamp))

bbox_centers_df = pd.DataFrame({
    'ImageName': df['ImageName'],
    'BBoxCenterLat': [lat for lat, lon, _ in bbox_centers_latlon],
    'BBoxCenterLon': [lon for lat, lon, _ in bbox_centers_latlon],
    'TimestampImage': [ts for _, _, ts in bbox_centers_latlon]
})

bbox_centers_df.to_excel('bbox_centers.xlsx', index=False)


# Creating txt files

Based on the bbox_centers excel creating txt files containing lat and lon of the labels

In [ ]:
from datetime import datetime
import os
import pandas as pd

save_path_karlo = 'C:/Users/Stella/AOSeR Dropbox/Stella Dumencic/Cres_experiment/work_folder/monitoring/target_detection'
save_path_stella = './detected_targets'

os.makedirs(save_path_stella, exist_ok=True)

bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

# Save each lat/lon pair to its own file
for index, row in bbox_centers_df.iterrows():
        filename = os.path.join(save_path, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename, 'w') as f:
            f.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}\n")

        filename_stella = os.path.join(save_path_stella, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename_stella, 'w') as f_stella:
            f_stella.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}\n")

# Converting the bounding box coordinates

Converting the bounding box coordinates from global to local

In [3]:
import numpy as np
from geo_converter import latlon_to_local, local_to_latlon
import pandas as pd

bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

base_point_test = [45.11703360020617, 14.1959855651055]

# Prepare array of all bbox center coordinates
latlon_points = bbox_centers_df[['BBoxCenterLat', 'BBoxCenterLon']].values

# Convert to local coordinates
drifter_location = latlon_to_local(latlon_points, base_point_test)

# Save to npz file with the required key and time_step
np.savez('bbox_centers_local.npz', time_step=9000, drifter_location=drifter_location)

# Advection of the targets

Continuously advects the location of the targets

In [ ]:
data = np.load('bbox_centers_local.npz')
print(data.files)
print(data['drifter_location'])
print(data['time_step'])

from marker_advection.advect_marker_locations import advect_marker_locations
import time

data_dir = f'C:/Users/Stella/Documents/cres/marker_advection/target_dir' # measurements in npz format
case_dir = f'C:/Users/Stella/Documents/cres/marker_advection/template_dir/case_Cres' # OpenFOAM case
advection_results_dir = f'C:/Users/Stella/Documents/cres/marker_advection/advection_results_dir' # advected marker locations
advection_time_step = 600 # desired advection time step

gaussian_noise=False

name = 0
advect_marker_locations(case_dir, advection_time_step, data_dir, advection_results_dir, name)
name = name + 1

['time_step', 'drifter_location']
[[ 3.61247011e+02  4.86264196e+01]
 [ 2.01516366e+02 -2.35948463e+01]
 [ 2.21603654e+02 -4.77361339e+00]
 [ 3.21840962e+02  9.33268060e+01]
 [ 2.90740806e+02  2.00401304e+01]
 [ 2.84533558e+02  4.46386154e+01]
 [ 2.72543385e+02  3.04378528e+01]
 [ 2.49797105e+02  2.59358144e+01]
 [ 2.25424616e+02  5.68923410e+01]
 [-1.69292360e+01  7.10113859e+01]
 [-5.82076609e-11  0.00000000e+00]
 [ 1.56804527e+02  6.34355364e+01]
 [ 1.18947513e+02  1.04052744e+02]
 [ 1.41216030e+01  1.23801192e+02]
 [ 9.93102528e+00  1.26714461e+02]
 [-2.51242726e+01  1.78638379e+02]
 [-7.18683490e+01  1.71636766e+02]
 [-4.34733060e+01  1.69559279e+02]
 [-2.43354476e+01  1.64791388e+02]
 [-1.94400889e+01  1.65648111e+02]
 [-4.73872527e+01  1.24683406e+02]
 [-8.63593534e+01  1.79585942e+02]
 [-8.57620594e+01  1.81091769e+02]
 [-9.11602224e+01  1.79583267e+02]
 [-1.27720167e+02  1.82820844e+02]]
9000


RuntimeError: No active exception to reraise

# Converting coordinates

Converts coordinates from local to global

In [ ]:
import numpy as np
from geo_converter import latlon_to_local, local_to_latlon

base_point_test = [45.11703360020617, 14.1959855651055]

# Prepare array of all bbox center coordinates
# Load advected marker locations from the npz file
data = np.load('marker_advection/advection_results_dir/advected_marker_locations_0.npz')
latlon_points = data['drifter_location']

# Convert to local coordinates
drifter_location = local_to_latlon(latlon_points, base_point_test)

# Save to npz file with the required key and time_step
np.savez('bbox_centers_global.npz', time_step=9000, drifter_location=drifter_location)

# Image map

In [6]:
import folium
import pandas as pd

# Load image info with lat/lon
info_df = pd.read_excel('image_info.xlsx')

# Center map at the mean location
center_lat = info_df['lat'].mean()
center_lon = info_df['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add markers for each image
for _, row in info_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['ImageName'],
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(m)

m.save('image_locations_map.html')

# Labels map

In [ ]:
import folium
import pandas as pd

# Load bbox center coordinates
bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

# Center the map at the mean location
center_lat = bbox_centers_df['BBoxCenterLat'].mean()
center_lon = bbox_centers_df['BBoxCenterLon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add markers for each bbox center
for _, row in bbox_centers_df.iterrows():
    folium.Marker(
        location=[row['BBoxCenterLat'], row['BBoxCenterLon']],
        popup=row['ImageName'],
        icon=folium.Icon(color='red', icon='crosshairs', prefix='fa')
    ).add_to(m)

m.save('bbox_centers_map.html')

# Test converting

In [ ]:
from geo_converter import latlon_to_local, local_to_latlon

base_point_test = [45.11703360020617, 14.1959855651055]
point = np.array([[44.952006, 14.364774]])

test_local_point = latlon_to_local(point, base_point_test)

print(test_local_point)

# Test npz file

Local coordinates

In [ ]:
data = np.load('marker_advection/advection_results_dir/advected_marker_locations_0.npz')
print(data.files)
print(data['drifter_location'])

# Test npz file

Global coordinates

In [ ]:
data = np.load('bbox_centers_global.npz')
print(data.files)
print(data['drifter_location'])